In [29]:
import pandas as pd
import numpy as np

In [30]:
from tqdm import tqdm

In [31]:
import collections

# Pre-processing BI data

In [32]:
path = './data/MosIndex_All.csv'
data = pd.read_csv(path, low_memory = False)

In [162]:
data.head()

,Date,County,Town,Village,VillageID,VillageLon,VillageLat,AreaType,HouseHold,InspectType,...,HIAeg,HILv,HILvAeg,CI,CILv,LI,LILv,Pupa,AI,Con100HH
0,2010/01/01,台北市,中正區,三愛里,6300500-031,121.530593083676,25.0358286563398,住宅,50,衛生所,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00
1,2010/01/01,嘉義市,西區,保安里,1002002-011,120.431350323348,23.4910599871089,住宅,156,衛生局,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00
2,2010/01/02,高雄市,鳳山區,生明里,6401200-045,120.36306711902,22.6176350677023,住宅,60,監測,...,3.333,1,1,5.714,3,0.000,0,0,0.0333,3.33
3,2010/01/02,台中市,大里區,塗城里,6602800-016,120.717131600963,24.0852343263972,住宅,50,衛生所,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00
4,2010/01/03,宜蘭縣,蘇澳鎮,存仁里,1000203-004,121.845912904116,24.6259386590972,住宅,60,衛生所,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00


In [163]:
county = data['County'].unique().tolist()
len(county)

22

In [164]:
county.remove('新竹市')
county.remove('嘉義市')

In [165]:
print(county)

['台北市', '高雄市', '台中市', '宜蘭縣', '台南市', '新北市', '彰化縣', '花蓮縣', '金門縣', '雲林縣', '新竹縣', '嘉義縣', '桃園市', '澎湖縣', '南投縣', '屏東縣', '台東縣', '基隆市', '連江縣', '苗栗縣']


In [166]:
use_cols = ['Date', 'County', 'BI', 'HI', 'CI']
bi_hi_ci = ['Date', 'BI', 'HI', 'CI']

In [167]:
data_distill = data[use_cols]

In [168]:
data_distill.head()

,Date,County,BI,HI,CI
0,2010/01/01,台北市,0.000,0.000,0.000
1,2010/01/01,嘉義市,0.000,0.000,0.000
2,2010/01/02,高雄市,3.333,3.333,5.714
3,2010/01/02,台中市,0.000,0.000,0.000
4,2010/01/03,宜蘭縣,0.000,0.000,0.000


In [169]:
# split data for 20 county

cnty_data = collections.defaultdict(list)

for i in tqdm(range(data_distill.shape[0])):
    row = data_distill.iloc[i, :].tolist()
    cnty = row[1]
    if cnty in county:
        #print(row)
        cnty_data[cnty].append(row)

100%|██████████| 444878/444878 [00:56<00:00, 7817.52it/s]


In [41]:
# convert data each county from list to dataframe

data_fin = collections.defaultdict()
for k, v in cnty_data.items():
    x = pd.DataFrame(cnty_data[k], columns = use_cols, dtype = float)
        
        #print(x.loc[x[c].isin(selected_words),c].value_counts())

        #print(x[c].value_counts())
    data_fin[k] = x

In [310]:
data_fin['高雄市'].tail()

,Date,County,BI,HI,CI
89405,2019/07/17,高雄市,36.000,14.000,58.064
89406,2019/07/17,高雄市,2.000,2.000,4.761
89407,2019/07/17,高雄市,0.000,0.000,0.000
89408,2019/07/17,高雄市,38.000,8.000,57.575
89409,2019/07/17,高雄市,2.255,2.255,18.750


In [318]:
# BI data

# replace 'None' with 'NaN' & groupby 'Date'

data_fin2 = {}


print('\n Count missing weeks of each county! \n')

def groupby_date(df):

    def mean_customized(x):
        #purpose: to calculate mean of a grouped column with mixed values, ex: [3, 6, 8, 'NaN', 'T', 9]
        #x: grouped column in data1
        #return: mean(x) where x is filtered out all 'str' values

        xx = x.astype(float)
        #tmp = [v for v in list(x) if isinstance(v, float)] # filter out 'str' in a grouped column
        tmp = [v for v in list(xx) if v != 1000] # filter out '1000' in a grouped column

        return np.mean(tmp)
    
    tm = df.replace(to_replace='None', value= 10000)

    tm['CI'] = tm.CI.astype(float)
    tm['BI'] = tm.BI.astype(float)
    tm['HI'] = tm.HI.astype(float)

    tm['Year'] = pd.to_datetime(tm['Date']).dt.year 

    tmm = tm[tm['Year'] >= 2012]

    tmm2 = tmm.groupby('Date', as_index=False).aggregate({'CI': mean_customized,
                                                         'BI': mean_customized,
                                                         'HI': mean_customized})
    return tmm2

for k in data_fin.keys():    

    df = data_fin[k]
    tmm = groupby_date(df)    

    tm2 = convert_date_to_yyyy_ww(tmm, date_week, end_time = '2019-05-31')  
    
    tm3 = bi_groupby_yyyy_ww(tm2)

    tm4 = bi_gen_full_time_table(tm3, week_total, county = k)

    data_fin2[k] = tm4



 Count missing weeks of each county! 

******************** Full Table:  花蓮縣 ******************** 
      YYYY_WW        BI       CI        HI County
382  2019_18  0.588889  2501.94  0.588889    花蓮縣
383  2019_19      0.94  577.961      0.94    花蓮縣
384  2019_20       0.4  2500.62       0.4    花蓮縣
385  2019_21  0.166667  4226.44  0.166667    花蓮縣
386  2019_22  0.171429  3333.54  0.171429    花蓮縣
******************** Full Table:  屏東縣 ******************** 
      YYYY_WW       BI       CI        HI County
382  2019_18  1.31313  2.50533  0.887226    屏東縣
383  2019_19  2.50175   4.4448   1.86839    屏東縣
384  2019_20  3.17047  7.67767   2.49549    屏東縣
385  2019_21  3.24846  5.37455   2.51807    屏東縣
386  2019_22  2.68822  5.52229    2.0062    屏東縣
******************** Full Table:  桃園市 ******************** 
      YYYY_WW       BI       CI        HI County
382  2019_18  1.45738  6.12008   1.45738    桃園市
383  2019_19  1.24774   1275.8  0.846742    桃園市
384  2019_20  1.56142  675.049   1.35442    桃園市
385

In [319]:
# Final step for BI, CI, HI
data_fin2['宜蘭縣'].tail()

,YYYY_WW,BI,CI,HI,County
382,2019_18,0.080125,333.681,0.151174,宜蘭縣
383,2019_19,0.349941,959.106,0.349941,宜蘭縣
384,2019_20,0.25825,1301.9,0.25825,宜蘭縣
385,2019_21,0.419875,2755.99,0.353208,宜蘭縣
386,2019_22,0.4592,4.23562,0.5592,宜蘭縣


In [317]:
def weather_convert_date_to_yyyy_ww(data_frame, yyyy_ww_dict):
    '''
    Input: data_frame
    Output: data_frame added yyyy_ww column
    '''
    yyww = []
    tmp = data_frame.Date
    date_key = list(tmp)
    
    for i in range(data_frame.shape[0]):
        
        k = date_key[i]
        try:
            yw = yyyy_ww_dict[k]
            #print(k, '-'*10, yw)
            yyww.append(yw)
        except: KeyError
            
        
    Year_Week = pd.Series(yyww)

    
    data_frame = data_frame.reset_index(drop = True)
    
    data_frame['Year_Week'] = Year_Week

    return data_frame


def convert_date_to_yyyy_ww(data_frame, yyyy_ww_dict, end_time = '2019-05-31'):
    '''
    Input: data_frame
    Output: data_frame added yyyy_ww column
    '''
    yyww = []
    
    tmp = data_frame.Date
    date_key = list(tmp)
    
    ey, em, ed = end_time.split('-')
    
    for i in range(data_frame.shape[0]):
        k = date_key[i]
        y, m, d = k.split('/')
        if y == ey and m == '06':
            break
        kk = y + '-' + m + '-' + d
        yw = yyyy_ww_dict[kk]
        yyww.append(yw)

    Year_Week = pd.Series(yyww)
    #print(Year_Week.shape)
    
    data_frame2 = data_frame.reset_index(drop = True)
    #print(data_frame2.shape)
    
    data_frame2['Year_Week'] = Year_Week
    return data_frame2

def groupby_yyyy_ww(data_frame, col = 'Year_Week'):
    
    def fill_nan_dummy(county_df):
        

        county_df = county_df.replace(to_replace='T', value=1000)
        county_df = county_df.replace(to_replace='X', value=1000)
        county_df = county_df.replace(to_replace='/', value=1000)
        county_df = county_df.replace(to_replace='...', value=1000)  

        #county_df = county_df.replace(to_replace='T', value=float('NaN'))
        #county_df = county_df.replace(to_replace='X', value=float('NaN'))
        #county_df = county_df.replace(to_replace='/', value=float('NaN'))
        #county_df = county_df.replace(to_replace='...', value=float('NaN'))
        county_df = county_df.astype(float)

        return county_df
    
    def mean_customized(x):
        #purpose: to calculate mean of a grouped column with mixed values, ex: [3, 6, 8, 'NaN', 'T', 9]
        #x: grouped column in data1
        #return: mean(x) where x is filtered out all 'str' values
        xx = fill_nan_dummy(x)
        #tmp = [v for v in list(x) if isinstance(v, float)] # filter out 'str' in a grouped column
        tmp = [v for v in list(xx) if v != 1000] # filter out '1000' in a grouped column

        
        return np.mean(tmp)


    # groupby & mean with condition & add customized function with groupby
    tmp8 = data_frame.groupby(col, as_index = False).aggregate({'Temperature': mean_customized,
                                                            'T Max': mean_customized,
                                                            'T Min': mean_customized,
                                                            'RH': mean_customized,
                                                            'Precp': mean_customized})    

    return tmp8
    
def bi_groupby_yyyy_ww(data_frame, col = 'Year_Week'):
    
    def fill_nan_dummy(county_df):
        

        county_df = county_df.replace(to_replace='None', value=1000)

        county_df = county_df.astype(float)

        return county_df
    
    def mean_customized(x):
        #purpose: to calculate mean of a grouped column with mixed values, ex: [3, 6, 8, 'NaN', 'T', 9]
        #x: grouped column in data1
        #return: mean(x) where x is filtered out all 'str' values
        
        xx = fill_nan_dummy(x)
        
        #tmp = [v for v in list(x) if isinstance(v, float)] # filter out 'str' in a grouped column
        tmp = [v for v in list(xx) if v != 1000] # filter out '1000' in a grouped column

        return np.mean(tmp)


    # groupby & mean with condition & add customized function with groupby
    tmp8 = data_frame.groupby(col, as_index = False).aggregate({'BI': mean_customized,
                                                            'CI': mean_customized,
                                                            'HI': mean_customized})    

    return tmp8

def bi_gen_full_time_table(data_frame, week_total, county = ''):
    
    '''
    Input: 
        data_frame: data with missing weeks of years
        week_total: total number of yyyy_ww 
    Output: 
        full_table: data_frame fulfilled missed yyyy_ww based on week_total
    
    '''
    
    full_table = []
    j = 0
    
    for i in range(len(week_total)):

        yw_ref = week_total[i]
        
        try:
            yw_cty = sorted(data_frame.Year_Week)[j]
        except: IndexError


        if yw_ref == yw_cty:
            bi = data_frame.BI[j]
            ci = data_frame.CI[j]
            hi = data_frame.HI[j]
            full_table.append([yw_ref, bi, ci, hi])
            j += 1
            continue

        else:
            full_table.append([yw_ref,'X', 'X', 'X']) # float('nan'), float('nan'), float('nan')])
            continue
            
    full_table = pd.DataFrame(full_table, columns = ['YYYY_WW', 'BI', 'CI', 'HI'])
    full_table['County'] = county
    print('*'*20, 'Full Table: ', county, '*'*20, '\n', full_table.tail())
    return full_table     

def find_missed_yyyy_ww(data_frame, week_total):
    a = set(data_frame.Year_Week)
    b = set(week_total)
    return b.difference(a)

In [228]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [238]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': ['T', 3, 4, 2, '/', 8],
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,T,5
1,B,3,0
2,C,4,3
3,A,2,3
4,B,/,7
5,C,8,9


In [268]:
def mean_customized(x):
    '''
    purpose: to calculate mean of a grouped column with mixed values, ex: [3, 6, 8, 'NaN', 'T', 9]
    x: grouped column in data1
    return: mean(x) where x is filtered out all 'str' values
    
    '''
    tmp = [v for v in list(x) if not isinstance(v, str)] # filter out 'str' in a grouped column

    return np.mean(tmp)

df.groupby('key').aggregate({'data1': filter_func,
                             'data2': 'mean'})


#df.groupby('key', as_index = False).agg([('Mean', filter_func)])

#display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

,data2,data1
key,,
A,4.0,2.0
B,3.5,3.0
C,6.0,6.0


## Generate YYYY_WW data standard

In [45]:
from datetime import date,timedelta


In [155]:
df = pd.DataFrame({'INSP_DATE2':pd.date_range(start = '2012-01-01', end = '2019-5-31')})

a =  df['INSP_DATE2'] - pd.offsets.Week(weekday=6)
b =  df['INSP_DATE2'] + pd.offsets.Week(weekday=5)

m1 = df['INSP_DATE2'] != (a + pd.offsets.Week())
m2 = df['INSP_DATE2'] != (b - pd.offsets.Week())

df['c'] = df['INSP_DATE2'].mask(m1, a)
df['d'] = df['INSP_DATE2'].mask(m2, b)

df

#for i in range(df.shape[0]):
#    print(df.loc[i], '\n')

,INSP_DATE2,c,d
0,2012-01-01,2012-01-01,2012-01-07
1,2012-01-02,2012-01-01,2012-01-07
2,2012-01-03,2012-01-01,2012-01-07
3,2012-01-04,2012-01-01,2012-01-07
4,2012-01-05,2012-01-01,2012-01-07
5,2012-01-06,2012-01-01,2012-01-07
6,2012-01-07,2012-01-01,2012-01-07
7,2012-01-08,2012-01-08,2012-01-14
8,2012-01-09,2012-01-08,2012-01-14
9,2012-01-10,2012-01-08,2012-01-14


In [47]:
rows = []
w = 1 # start week
y = 2012 # start year

# generate standard yyyy_ww

for b in tqdm(range(0, 2755, 7)):

    row = []
    win = df.iloc[b : (b+7), :]
    date = win.c
    #print(str(date).split('-'))
    #y, _, _ = str(list(date)[0]).split('-')
    
       
    if w <10:
        w_str = '0' + str(w)
    else:
        w_str = str(w)
    win["Year_Week"] = str(y) + '_' + str(w_str)
    
    if y != 2014 and w == 52:
        w = 0
        y += 1
    elif y == 2014 and w == 53:
        w = 0
        y += 1
    w += 1
    
    rows.append(win)

  0%|          | 0/394 [00:00<?, ?it/s]/home/basic/tensorflow/tensorflow_pkg/tf_1.12_cuda10.0/lib/python3.5/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 394/394 [00:19<00:00, 19.75it/s]


In [158]:
year_weekly = pd.concat(rows)
for i in range(year_weekly.shape[0]):
    print(year_weekly.loc[i])

INSP_DATE2    2012-01-01 00:00:00
c             2012-01-01 00:00:00
d             2012-01-07 00:00:00
Year_Week                 2012_01
Name: 0, dtype: object
INSP_DATE2    2012-01-02 00:00:00
c             2012-01-01 00:00:00
d             2012-01-07 00:00:00
Year_Week                 2012_01
Name: 1, dtype: object
INSP_DATE2    2012-01-03 00:00:00
c             2012-01-01 00:00:00
d             2012-01-07 00:00:00
Year_Week                 2012_01
Name: 2, dtype: object
INSP_DATE2    2012-01-04 00:00:00
c             2012-01-01 00:00:00
d             2012-01-07 00:00:00
Year_Week                 2012_01
Name: 3, dtype: object
INSP_DATE2    2012-01-05 00:00:00
c             2012-01-01 00:00:00
d             2012-01-07 00:00:00
Year_Week                 2012_01
Name: 4, dtype: object
INSP_DATE2    2012-01-06 00:00:00
c             2012-01-01 00:00:00
d             2012-01-07 00:00:00
Year_Week                 2012_01
Name: 5, dtype: object
INSP_DATE2    2012-01-07 00:00:00
c           

Name: 359, dtype: object
INSP_DATE2    2012-12-26 00:00:00
c             2012-12-23 00:00:00
d             2012-12-29 00:00:00
Year_Week                 2012_52
Name: 360, dtype: object
INSP_DATE2    2012-12-27 00:00:00
c             2012-12-23 00:00:00
d             2012-12-29 00:00:00
Year_Week                 2012_52
Name: 361, dtype: object
INSP_DATE2    2012-12-28 00:00:00
c             2012-12-23 00:00:00
d             2012-12-29 00:00:00
Year_Week                 2012_52
Name: 362, dtype: object
INSP_DATE2    2012-12-29 00:00:00
c             2012-12-23 00:00:00
d             2012-12-29 00:00:00
Year_Week                 2012_52
Name: 363, dtype: object
INSP_DATE2    2012-12-30 00:00:00
c             2012-12-30 00:00:00
d             2013-01-05 00:00:00
Year_Week                 2013_01
Name: 364, dtype: object
INSP_DATE2    2012-12-31 00:00:00
c             2012-12-30 00:00:00
d             2013-01-05 00:00:00
Year_Week                 2013_01
Name: 365, dtype: object
INSP_DATE

Name: 794, dtype: object
INSP_DATE2    2014-03-06 00:00:00
c             2014-03-02 00:00:00
d             2014-03-08 00:00:00
Year_Week                 2014_10
Name: 795, dtype: object
INSP_DATE2    2014-03-07 00:00:00
c             2014-03-02 00:00:00
d             2014-03-08 00:00:00
Year_Week                 2014_10
Name: 796, dtype: object
INSP_DATE2    2014-03-08 00:00:00
c             2014-03-02 00:00:00
d             2014-03-08 00:00:00
Year_Week                 2014_10
Name: 797, dtype: object
INSP_DATE2    2014-03-09 00:00:00
c             2014-03-09 00:00:00
d             2014-03-15 00:00:00
Year_Week                 2014_11
Name: 798, dtype: object
INSP_DATE2    2014-03-10 00:00:00
c             2014-03-09 00:00:00
d             2014-03-15 00:00:00
Year_Week                 2014_11
Name: 799, dtype: object
INSP_DATE2    2014-03-11 00:00:00
c             2014-03-09 00:00:00
d             2014-03-15 00:00:00
Year_Week                 2014_11
Name: 800, dtype: object
INSP_DATE

Name: 1210, dtype: object
INSP_DATE2    2015-04-26 00:00:00
c             2015-04-26 00:00:00
d             2015-05-02 00:00:00
Year_Week                 2015_17
Name: 1211, dtype: object
INSP_DATE2    2015-04-27 00:00:00
c             2015-04-26 00:00:00
d             2015-05-02 00:00:00
Year_Week                 2015_17
Name: 1212, dtype: object
INSP_DATE2    2015-04-28 00:00:00
c             2015-04-26 00:00:00
d             2015-05-02 00:00:00
Year_Week                 2015_17
Name: 1213, dtype: object
INSP_DATE2    2015-04-29 00:00:00
c             2015-04-26 00:00:00
d             2015-05-02 00:00:00
Year_Week                 2015_17
Name: 1214, dtype: object
INSP_DATE2    2015-04-30 00:00:00
c             2015-04-26 00:00:00
d             2015-05-02 00:00:00
Year_Week                 2015_17
Name: 1215, dtype: object
INSP_DATE2    2015-05-01 00:00:00
c             2015-04-26 00:00:00
d             2015-05-02 00:00:00
Year_Week                 2015_17
Name: 1216, dtype: object
IN

Name: 1560, dtype: object
INSP_DATE2    2016-04-10 00:00:00
c             2016-04-10 00:00:00
d             2016-04-16 00:00:00
Year_Week                 2016_15
Name: 1561, dtype: object
INSP_DATE2    2016-04-11 00:00:00
c             2016-04-10 00:00:00
d             2016-04-16 00:00:00
Year_Week                 2016_15
Name: 1562, dtype: object
INSP_DATE2    2016-04-12 00:00:00
c             2016-04-10 00:00:00
d             2016-04-16 00:00:00
Year_Week                 2016_15
Name: 1563, dtype: object
INSP_DATE2    2016-04-13 00:00:00
c             2016-04-10 00:00:00
d             2016-04-16 00:00:00
Year_Week                 2016_15
Name: 1564, dtype: object
INSP_DATE2    2016-04-14 00:00:00
c             2016-04-10 00:00:00
d             2016-04-16 00:00:00
Year_Week                 2016_15
Name: 1565, dtype: object
INSP_DATE2    2016-04-15 00:00:00
c             2016-04-10 00:00:00
d             2016-04-16 00:00:00
Year_Week                 2016_15
Name: 1566, dtype: object
IN

Name: 1992, dtype: object
INSP_DATE2    2017-06-16 00:00:00
c             2017-06-11 00:00:00
d             2017-06-17 00:00:00
Year_Week                 2017_24
Name: 1993, dtype: object
INSP_DATE2    2017-06-17 00:00:00
c             2017-06-11 00:00:00
d             2017-06-17 00:00:00
Year_Week                 2017_24
Name: 1994, dtype: object
INSP_DATE2    2017-06-18 00:00:00
c             2017-06-18 00:00:00
d             2017-06-24 00:00:00
Year_Week                 2017_25
Name: 1995, dtype: object
INSP_DATE2    2017-06-19 00:00:00
c             2017-06-18 00:00:00
d             2017-06-24 00:00:00
Year_Week                 2017_25
Name: 1996, dtype: object
INSP_DATE2    2017-06-20 00:00:00
c             2017-06-18 00:00:00
d             2017-06-24 00:00:00
Year_Week                 2017_25
Name: 1997, dtype: object
INSP_DATE2    2017-06-21 00:00:00
c             2017-06-18 00:00:00
d             2017-06-24 00:00:00
Year_Week                 2017_25
Name: 1998, dtype: object
IN

Name: 2598, dtype: object
INSP_DATE2    2019-02-12 00:00:00
c             2019-02-10 00:00:00
d             2019-02-16 00:00:00
Year_Week                 2019_07
Name: 2599, dtype: object
INSP_DATE2    2019-02-13 00:00:00
c             2019-02-10 00:00:00
d             2019-02-16 00:00:00
Year_Week                 2019_07
Name: 2600, dtype: object
INSP_DATE2    2019-02-14 00:00:00
c             2019-02-10 00:00:00
d             2019-02-16 00:00:00
Year_Week                 2019_07
Name: 2601, dtype: object
INSP_DATE2    2019-02-15 00:00:00
c             2019-02-10 00:00:00
d             2019-02-16 00:00:00
Year_Week                 2019_07
Name: 2602, dtype: object
INSP_DATE2    2019-02-16 00:00:00
c             2019-02-10 00:00:00
d             2019-02-16 00:00:00
Year_Week                 2019_07
Name: 2603, dtype: object
INSP_DATE2    2019-02-17 00:00:00
c             2019-02-17 00:00:00
d             2019-02-23 00:00:00
Year_Week                 2019_08
Name: 2604, dtype: object
IN

In [179]:
# unique week
week_total = year_weekly.Year_Week.unique()
for w in week_total:
    print(w)

2012_01
2012_02
2012_03
2012_04
2012_05
2012_06
2012_07
2012_08
2012_09
2012_10
2012_11
2012_12
2012_13
2012_14
2012_15
2012_16
2012_17
2012_18
2012_19
2012_20
2012_21
2012_22
2012_23
2012_24
2012_25
2012_26
2012_27
2012_28
2012_29
2012_30
2012_31
2012_32
2012_33
2012_34
2012_35
2012_36
2012_37
2012_38
2012_39
2012_40
2012_41
2012_42
2012_43
2012_44
2012_45
2012_46
2012_47
2012_48
2012_49
2012_50
2012_51
2012_52
2013_01
2013_02
2013_03
2013_04
2013_05
2013_06
2013_07
2013_08
2013_09
2013_10
2013_11
2013_12
2013_13
2013_14
2013_15
2013_16
2013_17
2013_18
2013_19
2013_20
2013_21
2013_22
2013_23
2013_24
2013_25
2013_26
2013_27
2013_28
2013_29
2013_30
2013_31
2013_32
2013_33
2013_34
2013_35
2013_36
2013_37
2013_38
2013_39
2013_40
2013_41
2013_42
2013_43
2013_44
2013_45
2013_46
2013_47
2013_48
2013_49
2013_50
2013_51
2013_52
2014_01
2014_02
2014_03
2014_04
2014_05
2014_06
2014_07
2014_08
2014_09
2014_10
2014_11
2014_12
2014_13
2014_14
2014_15
2014_16
2014_17
2014_18
2014_19
2014_20
2014_21


In [154]:
# yyyy_mm_dd: yyyy_ww
date_week = collections.defaultdict()
for i in range(year_weekly.shape[0]):
    date_key = year_weekly.INSP_DATE2[i]
    k = str(date_key).split(' ')[0]
    print(k)
    week_val = year_weekly.Year_Week[i]
    date_week[k] = week_val

2012-01-01
2012-01-02
2012-01-03
2012-01-04
2012-01-05
2012-01-06
2012-01-07
2012-01-08
2012-01-09
2012-01-10
2012-01-11
2012-01-12
2012-01-13
2012-01-14
2012-01-15
2012-01-16
2012-01-17
2012-01-18
2012-01-19
2012-01-20
2012-01-21
2012-01-22
2012-01-23
2012-01-24
2012-01-25
2012-01-26
2012-01-27
2012-01-28
2012-01-29
2012-01-30
2012-01-31
2012-02-01
2012-02-02
2012-02-03
2012-02-04
2012-02-05
2012-02-06
2012-02-07
2012-02-08
2012-02-09
2012-02-10
2012-02-11
2012-02-12
2012-02-13
2012-02-14
2012-02-15
2012-02-16
2012-02-17
2012-02-18
2012-02-19
2012-02-20
2012-02-21
2012-02-22
2012-02-23
2012-02-24
2012-02-25
2012-02-26
2012-02-27
2012-02-28
2012-02-29
2012-03-01
2012-03-02
2012-03-03
2012-03-04
2012-03-05
2012-03-06
2012-03-07
2012-03-08
2012-03-09
2012-03-10
2012-03-11
2012-03-12
2012-03-13
2012-03-14
2012-03-15
2012-03-16
2012-03-17
2012-03-18
2012-03-19
2012-03-20
2012-03-21
2012-03-22
2012-03-23
2012-03-24
2012-03-25
2012-03-26
2012-03-27
2012-03-28
2012-03-29
2012-03-30
2012-03-31

2017-11-28
2017-11-29
2017-11-30
2017-12-01
2017-12-02
2017-12-03
2017-12-04
2017-12-05
2017-12-06
2017-12-07
2017-12-08
2017-12-09
2017-12-10
2017-12-11
2017-12-12
2017-12-13
2017-12-14
2017-12-15
2017-12-16
2017-12-17
2017-12-18
2017-12-19
2017-12-20
2017-12-21
2017-12-22
2017-12-23
2017-12-24
2017-12-25
2017-12-26
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2017-12-31
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-13
2018-01-14
2018-01-15
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-20
2018-01-21
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-27
2018-01-28
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-03
2018-02-04
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-10
2018-02-11
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-17
2018-02-18
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-24
2018-02-25
2018-02-26

In [51]:
# find missing week
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 


In [177]:
for k, v in sorted(date_week.items()):
    print(k, '-', v)

2012-01-01 - 2012_01
2012-01-02 - 2012_01
2012-01-03 - 2012_01
2012-01-04 - 2012_01
2012-01-05 - 2012_01
2012-01-06 - 2012_01
2012-01-07 - 2012_01
2012-01-08 - 2012_02
2012-01-09 - 2012_02
2012-01-10 - 2012_02
2012-01-11 - 2012_02
2012-01-12 - 2012_02
2012-01-13 - 2012_02
2012-01-14 - 2012_02
2012-01-15 - 2012_03
2012-01-16 - 2012_03
2012-01-17 - 2012_03
2012-01-18 - 2012_03
2012-01-19 - 2012_03
2012-01-20 - 2012_03
2012-01-21 - 2012_03
2012-01-22 - 2012_04
2012-01-23 - 2012_04
2012-01-24 - 2012_04
2012-01-25 - 2012_04
2012-01-26 - 2012_04
2012-01-27 - 2012_04
2012-01-28 - 2012_04
2012-01-29 - 2012_05
2012-01-30 - 2012_05
2012-01-31 - 2012_05
2012-02-01 - 2012_05
2012-02-02 - 2012_05
2012-02-03 - 2012_05
2012-02-04 - 2012_05
2012-02-05 - 2012_06
2012-02-06 - 2012_06
2012-02-07 - 2012_06
2012-02-08 - 2012_06
2012-02-09 - 2012_06
2012-02-10 - 2012_06
2012-02-11 - 2012_06
2012-02-12 - 2012_07
2012-02-13 - 2012_07
2012-02-14 - 2012_07
2012-02-15 - 2012_07
2012-02-16 - 2012_07
2012-02-17 - 

2015-04-07 - 2015_14
2015-04-08 - 2015_14
2015-04-09 - 2015_14
2015-04-10 - 2015_14
2015-04-11 - 2015_14
2015-04-12 - 2015_15
2015-04-13 - 2015_15
2015-04-14 - 2015_15
2015-04-15 - 2015_15
2015-04-16 - 2015_15
2015-04-17 - 2015_15
2015-04-18 - 2015_15
2015-04-19 - 2015_16
2015-04-20 - 2015_16
2015-04-21 - 2015_16
2015-04-22 - 2015_16
2015-04-23 - 2015_16
2015-04-24 - 2015_16
2015-04-25 - 2015_16
2015-04-26 - 2015_17
2015-04-27 - 2015_17
2015-04-28 - 2015_17
2015-04-29 - 2015_17
2015-04-30 - 2015_17
2015-05-01 - 2015_17
2015-05-02 - 2015_17
2015-05-03 - 2015_18
2015-05-04 - 2015_18
2015-05-05 - 2015_18
2015-05-06 - 2015_18
2015-05-07 - 2015_18
2015-05-08 - 2015_18
2015-05-09 - 2015_18
2015-05-10 - 2015_19
2015-05-11 - 2015_19
2015-05-12 - 2015_19
2015-05-13 - 2015_19
2015-05-14 - 2015_19
2015-05-15 - 2015_19
2015-05-16 - 2015_19
2015-05-17 - 2015_20
2015-05-18 - 2015_20
2015-05-19 - 2015_20
2015-05-20 - 2015_20
2015-05-21 - 2015_20
2015-05-22 - 2015_20
2015-05-23 - 2015_20
2015-05-24 - 

2018-06-17 - 2018_25
2018-06-18 - 2018_25
2018-06-19 - 2018_25
2018-06-20 - 2018_25
2018-06-21 - 2018_25
2018-06-22 - 2018_25
2018-06-23 - 2018_25
2018-06-24 - 2018_26
2018-06-25 - 2018_26
2018-06-26 - 2018_26
2018-06-27 - 2018_26
2018-06-28 - 2018_26
2018-06-29 - 2018_26
2018-06-30 - 2018_26
2018-07-01 - 2018_27
2018-07-02 - 2018_27
2018-07-03 - 2018_27
2018-07-04 - 2018_27
2018-07-05 - 2018_27
2018-07-06 - 2018_27
2018-07-07 - 2018_27
2018-07-08 - 2018_28
2018-07-09 - 2018_28
2018-07-10 - 2018_28
2018-07-11 - 2018_28
2018-07-12 - 2018_28
2018-07-13 - 2018_28
2018-07-14 - 2018_28
2018-07-15 - 2018_29
2018-07-16 - 2018_29
2018-07-17 - 2018_29
2018-07-18 - 2018_29
2018-07-19 - 2018_29
2018-07-20 - 2018_29
2018-07-21 - 2018_29
2018-07-22 - 2018_30
2018-07-23 - 2018_30
2018-07-24 - 2018_30
2018-07-25 - 2018_30
2018-07-26 - 2018_30
2018-07-27 - 2018_30
2018-07-28 - 2018_30
2018-07-29 - 2018_31
2018-07-30 - 2018_31
2018-07-31 - 2018_31
2018-08-01 - 2018_31
2018-08-02 - 2018_31
2018-08-03 - 

# Pre-processing Weather data

In [63]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [64]:
mypath = './data/tw_weekly_weather/'
listdir(mypath)

['Miaoli',
 'Tainan',
 'Penghu',
 'Chiayi',
 'Taichung',
 'Hsinchu',
 'Nantou',
 'Pingtung',
 'Changhua',
 'Lienchang',
 'Taoyuan',
 'Taitung',
 'Kinmen',
 'New Taipei',
 'Yunlin',
 'Hualien',
 'Keelung',
 'Yilan',
 'Kaohsiung',
 'Taipei']

In [181]:
folders = [mypath + folder + '/' for folder in listdir(mypath)]
print(folders)

['./data/tw_weekly_weather/Miaoli/', './data/tw_weekly_weather/Tainan/', './data/tw_weekly_weather/Penghu/', './data/tw_weekly_weather/Chiayi/', './data/tw_weekly_weather/Taichung/', './data/tw_weekly_weather/Hsinchu/', './data/tw_weekly_weather/Nantou/', './data/tw_weekly_weather/Pingtung/', './data/tw_weekly_weather/Changhua/', './data/tw_weekly_weather/Lienchang/', './data/tw_weekly_weather/Taoyuan/', './data/tw_weekly_weather/Taitung/', './data/tw_weekly_weather/Kinmen/', './data/tw_weekly_weather/New Taipei/', './data/tw_weekly_weather/Yunlin/', './data/tw_weekly_weather/Hualien/', './data/tw_weekly_weather/Keelung/', './data/tw_weekly_weather/Yilan/', './data/tw_weekly_weather/Kaohsiung/', './data/tw_weekly_weather/Taipei/']


In [182]:
from collections import defaultdict

folders_files = defaultdict(list)
for folder in folders:
    folders_files[folder] = listdir(folder)

### Get final preprocessed for Weather


In [299]:

all_county_weather = collections.defaultdict()

print('\n Count missing weeks of weather \n')

for key_county, value_files in folders_files.items():
    
    cty_name = key_county.split('/')[-2]
    if cty_name == 'Kinmen' or cty_name == 'Lienchang':
        continue
    #print(cty_name, '\n')
    
    county = add_date_to_county(key_county, cut_year = 2012)
            

        
    # fill nan, need to change
    #cty = fill_nan_dummy(county)
    cty = county
    
    cty1 = weather_convert_date_to_yyyy_ww(cty, date_week)
    
    print('\n', '*' * 20, 'Cty name = ', cty_name, '*'*20)
    #print('\n', cty1.tail())    
    #print(find_missed_yyyy_ww(cty1, week_total))
    # groupby week
    cty2 = groupby_yyyy_ww(cty1, col = 'Year_Week')
    
    print('\n', cty2.tail())    

    
    # count missing data
    #selected_words = [float('NaN')]

    #for c in cty2:
    #    print('\n', cty2.loc[cty2[c].isin(selected_words),c].value_counts())
    
    all_county_weather[cty_name] = cty2


 Count missing weeks of weather 


 ******************** Cty name =  Taichung ********************

     Year_Week  Temperature         RH      T Max      T Min      Precp
382   2019_18    24.885714  78.285714  29.242857  22.100000  18.428571
383   2019_19    23.414286  85.571429  27.414286  20.771429  10.000000
384   2019_20    27.657143  80.714286  31.671429  24.742857  12.357143
385   2019_21    25.400000  84.142857  29.485714  22.914286  36.666667
386   2019_22    26.850000  78.500000  31.250000  23.433333   3.083333

 ******************** Cty name =  Yilan ********************

     Year_Week  Temperature         RH      T Max      T Min      Precp
382   2019_18    23.285714  83.285714  27.071429  20.585714  12.700000
383   2019_19    21.757143  87.714286  24.714286  19.700000  17.066667
384   2019_20    26.200000  87.428571  30.614286  23.657143  20.871429
385   2019_21    24.285714  80.000000  27.614286  21.642857   3.314286
386   2019_22    25.000000  81.833333  29.433333  22.


 ******************** Cty name =  Tainan ********************

     Year_Week  Temperature         RH      T Max      T Min      Precp
382   2019_18    26.071429  73.285714  29.485714  23.400000   7.142857
383   2019_19    24.571429  78.714286  28.171429  21.928571   3.642857
384   2019_20    29.057143  71.285714  32.528571  26.628571   3.500000
385   2019_21    27.514286  75.000000  30.900000  24.885714  10.250000
386   2019_22    28.150000  72.666667  31.750000  25.066667   1.666667


In [300]:
# example: Miaoli weather
all_county_weather['Taichung'].tail()

,Year_Week,Temperature,RH,T Max,T Min,Precp
382,2019_18,24.885714,78.285714,29.242857,22.100000,18.428571
383,2019_19,23.414286,85.571429,27.414286,20.771429,10.000000
384,2019_20,27.657143,80.714286,31.671429,24.742857,12.357143
385,2019_21,25.400000,84.142857,29.485714,22.914286,36.666667
386,2019_22,26.850000,78.500000,31.250000,23.433333,3.083333


In [322]:
compiled = []
final_weather_dict = {}
for cty, cty_df in all_county_weather.items():
    cty_df['County'] = cty
    cty_df2 = cty_df[['County', 'Year_Week', 'Temperature', 'RH', 'T Max', 'T Min', 'Precp']]
    compiled.append(cty_df2)
    final_weather_dict[cty] = cty_df2
final_weather = pd.concat(compiled, axis = 0, ignore_index = True)

In [308]:
save_path = './data/final_weather.csv'
final_weather.to_csv(save_path)

In [320]:
# example: Miaoli BI, CI, HI
data_fin2[cty_en_cn['Miaoli']].tail()

,YYYY_WW,BI,CI,HI,County
382,2019_18,2.99017,844.778,2.5735,苗栗縣
383,2019_19,4,26.19,3,苗栗縣
384,2019_20,3,15.838,3.16667,苗栗縣
385,2019_21,3.4,13.6534,2.73333,苗栗縣
386,2019_22,2.92255,296.866,2.52255,苗栗縣


### Compile Weather + BI data

In [323]:
full_data = []
for cty_cn, cty_en in cty_cn_en.items():
    bi = data_fin2[cty_cn]
    te = final_weather_dict[cty_en]
    tmp = [bi, te]
    cty_bi_te = pd.concat(tmp, axis = 1) # concat column-by-column & keep old column names
    full_data.append(cty_bi_te)
    

In [324]:
full_data2 = pd.concat(full_data, axis = 0, ignore_index = True) # concat row-by-row & auto generate new index

In [325]:
full_data2.shape

(6966, 12)

# FINAL PREPROCESSED DATA (FULL 18 COUNTY + WEATHER + BI, CI, HI)

In [326]:
full_data2.head()

,YYYY_WW,BI,CI,HI,County,County,Year_Week,Temperature,RH,T Max,T Min,Precp
0,2012_01,0.5,4668.45,0.5,花蓮縣,Hualien,2012_01,16.614286,75.714286,18.657143,14.714286,0.571429
1,2012_02,0,750,0,花蓮縣,Hualien,2012_02,18.942857,75.428571,21.557143,16.800000,0.357143
2,2012_03,0,833.333,0,花蓮縣,Hualien,2012_03,19.300000,76.428571,21.914286,17.100000,2.400000
3,2012_04,X,X,X,花蓮縣,Hualien,2012_04,16.142857,79.714286,18.600000,13.771429,5.000000
4,2012_05,0,0,0,花蓮縣,Hualien,2012_05,17.957143,76.571429,20.414286,16.057143,1.000000


In [332]:
write_data = full_data2[['County', 'YYYY_WW', 'BI', 'CI', 'HI', 'Temperature', 'T Max', 'T Min', 'RH', 'Precp']]
write_data.head()
write_data.to_csv('./data/fullhouse.csv')

#### Utils

In [5]:
def add_date_to_county(folder_path, cut_year = 2012):
    county = [] # one county
    
    tmp = sorted(list(folders_files[folder_path]))
    for file in tmp:
        file_path = folder_path + file
        tmp_data = pd.read_csv(file_path)
        
        _, yyyy, mm = file.split('.')[0].split('-')
        yyyy_mm = yyyy + '-' + mm
        
        tmp2 = list(tmp_data.ObsTime)
        tmp3 = []
        for day in tmp2:
            if day < 10:
                day_str = '0'+ str(day)
            else:
                day_str = str(day)
            yyyy_mm_dd = yyyy_mm + '-' + day_str
            
            tmp3.append(yyyy_mm_dd)
            
        tmp_data['Date'] = tmp3
        
        use_cols = ['Temperature', 'T Max', 'T Min', 'RH', 'Precp', 'Date']
        
        tmp_data2 = tmp_data[use_cols]
        
        county.append(tmp_data2)
        
        county_df = pd.concat(county)
        
        # select year
        county_df['Year'] = pd.to_datetime(county_df['Date']).dt.year

        county_df = county_df[county_df['Year'] >= cut_year]
        
    return county_df
        

In [199]:
def fill_nan_dummy(county_df):
        

    #county_df = county_df.replace(to_replace='T', value=0)
    #county_df = county_df.replace(to_replace='X', value=0)
    #county_df = county_df.replace(to_replace='/', value=0)
    #county_df = county_df.replace(to_replace='...', value=0)  
    
    #county_df = county_df.replace(to_replace='T', value=float('NaN'))
    #county_df = county_df.replace(to_replace='X', value=float('NaN'))
    #county_df = county_df.replace(to_replace='/', value=float('NaN'))
    #county_df = county_df.replace(to_replace='...', value=float('NaN'))
    
    #county_df = county_df.replace(to_replace= 'inf', value=float('NaN'))

    county_df['Temperature'] = county_df['Temperature'].astype(float)
    county_df['T Max'] = county_df['T Max'].astype(float)
    county_df['T Min'] = county_df['T Min'].astype(float)
    county_df['Precp'] = county_df['Precp'].astype(float)
    county_df['RH'] = county_df['RH'].astype(float)

    
    return county_df


def fill_nan(county_df):
        

        
    county_df = county_df.replace(to_replace='T', value=float('NaN'))
    county_df = county_df.replace(to_replace='X', value=float('NaN'))
    county_df = county_df.replace(to_replace='/', value=float('NaN'))
    county_df = county_df.replace(to_replace='...', value=float('NaN'))
    #county_df = county_df.replace(to_replace= 'inf', value=float('NaN'))

    county_df['Temperature'] = county_df['Temperature'].astype(float)
    county_df['T Max'] = county_df['T Max'].astype(float)
    county_df['T Min'] = county_df['T Min'].astype(float)
    county_df['Precp'] = county_df['Precp'].astype(float)
    county_df['RH'] = county_df['RH'].astype(float)

    #data_x2 = pd.DataFrame(data_x, columns = weather_cols, dtype = float)

    #df['feature_count'].fillna(df['client_name'].map(s), inplace=True)
    county_df['Temperature'] = county_df['Temperature'].fillna(county_df['Temperature'].mean())
    
    county_df['T Max'] = county_df['T Max'].fillna(county_df['T Max'].mean())
    county_df['T Min'] = county_df['T Min'].fillna(county_df['T Min'].mean())
    county_df['Precp'] = county_df['Precp'].fillna(county_df['Precp'].mean()) 
    county_df['RH'] = county_df['RH'].fillna(county_df['RH'].mean()) 
    
    return county_df

In [173]:
# date_week dict, ex: 2019-5-31: 2019_22
new_date_week = {}
for k, v in date_week.items():
    yyyy_mm_dd = str((k)).split(' ')
    new_date_week[yyyy_mm_dd] = v

TypeError: unhashable type: 'list'

In [8]:
def weather_convert_date_to_yyyy_ww(data_frame, yyyy_ww_dict):
    '''
    Input: data_frame
    Output: data_frame added yyyy_ww column
    '''
    yyww = []
    tmp = data_frame.Date
    date_key = list(tmp)
    
    for i in range(data_frame.shape[0]):
        
        k = date_key[i]
        try:
            yw = yyyy_ww_dict[k]
            #print(k, '-'*10, yw)
            yyww.append(yw)
        except: KeyError
            
        
    Year_Week = pd.Series(yyww)

    
    data_frame = data_frame.reset_index(drop = True)
    
    data_frame['Year_Week'] = Year_Week

    return data_frame

In [9]:
def weather_gen_full_time_table(data_frame, week_total, county = ''):
    '''
    Input: 
        data_frame: data with missing weeks of years
        week_total: total number of yyyy_ww 
    Output: 
        full_table: data_frame fulfilled missed yyyy_ww based on week_total
    
    '''
    
    full_table = []
    j = 0
    for i in tqdm(range(len(week_total))):
        #print(i)

        yw_ref = week_total[i]
        
        try:
            yw_cty = sorted(data_frame.Year_Week)[j]
        except: IndexError
            
        #print(yw_cty)
        #print(j, yw_ref, yw_cty)

        if yw_ref == yw_cty:
            temp = data_frame['Temperature'][j]
            tmax = data_frame['T Max'][j]
            tmin = data_frame['T Min'][j]
            rh = data_frame['RH'][j]
            recp = data_frame['Precp'][j]

            full_table.append([yw_ref, temp, tmax, tmin, rh, recp])
            j += 1
            continue

        else:
            full_table.append([yw_ref, float('nan'), float('nan'), float('nan')])
            continue
            
    full_table = pd.DataFrame(full_table, columns = ['YYYY_WW', 'Temperature', 'T Max', 'T Min', 'RH', 'Precp'])
    full_table['County'] = county
    
    return full_table   

In [14]:
# TW county in Chinese English dict
cty_chinese = ['台北市', '高雄市', '台中市', '宜蘭縣', '台南市', 
               '新北市', '彰化縣', '花蓮縣', '雲林縣', 
               '新竹縣', '嘉義縣', '桃園市', '澎湖縣', '南投縣', 
               '屏東縣', '台東縣', '基隆市', '苗栗縣'] # '連江縣', '金門縣'

cty_english = ['Taipei', 'Kaohsiung', 'Taichung', 'Yilan',  'Tainan',
               'New Taipei', 'Changhua', 'Hualien', 'Yunlin',
               'Hsinchu',  'Chiayi',  'Taoyuan',  'Penghu',  'Nantou',
               'Pingtung', 'Taitung',  'Keelung', 'Miaoli']

cty_cn_en = collections.defaultdict()
cty_en_cn = collections.defaultdict()

# create 2 dict
for cn, en in zip(cty_chinese, cty_english):
    cty_cn_en[cn] = en
    cty_en_cn[en] = cn


##### END